# Simple tutorial on how to run dCache in k8s

This tutorial will guide you through the process of deploying dCache in a k8s cluster. 
We will use the official dCache docker images and helm charts to deploy the dCache services.


> Notes to notebook: This notebook requires bash kernel that can be installed, if needed:
```bash
pip install bash_kernel
python -m bash_kernel.install
```

The 3rd-party dependencies as zookeeper, postgresql and kafka started in k8s too.

## Prerequisites

- A running k8s cluster
- Helm installed
- kubectl installed

## Create a namespace

In [16]:
PATH=$PATH:~/bin
export K8S_NAMESPACE=dcache-test
export KUBECONFIG=~/.kube/config
kubectl create namespace ${K8S_NAMESPACE}

namespace/dcache-test created


In [15]:
kubectl get ns

NAME                          STATUS   AGE
cattle-fleet-system           Active   46d
cattle-impersonation-system   Active   46d
cattle-system                 Active   46d
ceph-csi-rbd                  Active   43d
dcache-build-117653           Active   24h
dcache-dev-infrastructure     Active   43d
dcache-gitlab-runner          Active   43d
default                       Active   46d
kube-node-lease               Active   46d
kube-public                   Active   46d
kube-system                   Active   46d
local                         Active   46d
tigran-test                   Active   24h


## Add Helm repositories

In [17]:
helm repo add bitnami https://charts.bitnami.com/bitnami
helm repo add dcache https://gitlab.desy.de/api/v4/projects/7648/packages/helm/test
helm repo update

"bitnami" has been added to your repositories
"dcache" has been added to your repositories
Hang tight while we grab the latest from your chart repositories...
...Successfully got an update from the "dcache" chart repository
...Successfully got an update from the "bitnami" chart repository
Update Complete. ⎈Happy Helming!⎈


## Deploy external services

Pinpoint PostgreSQL and Kafka to specific versions to avoid compatibility issues.

> Despite the fact, that dCache wotks with other versions of PostgreSQL and Kafka, the _helm_ charts are incompatible.

In [18]:
helm -n ${K8S_NAMESPACE} install --wait --set auth.username=dcache \
                --set auth.password=let-me-in --set auth.database=chimera  \
                chimera bitnami/postgresql --version=12.12.10

helm -n ${K8S_NAMESPACE} install --wait cells bitnami/zookeeper
helm -n ${K8S_NAMESPACE} install --wait --set externalZookeeper.servers=cells-zookeeper \
                 --set kraft.enabled=false billing bitnami/kafka --version 23.0.7

NAME: chimera
LAST DEPLOYED: Tue Jun  4 21:52:15 2024
NAMESPACE: dcache-test
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
CHART NAME: postgresql
CHART VERSION: 12.12.10
APP VERSION: 15.4.0

** Please be patient while the chart is being deployed **

PostgreSQL can be accessed via port 5432 on the following DNS names from within your cluster:

    chimera-postgresql.dcache-test.svc.cluster.local - Read/Write connection

To get the password for "postgres" run:

    export POSTGRES_ADMIN_PASSWORD=$(kubectl get secret --namespace dcache-test chimera-postgresql -o jsonpath="{.data.postgres-password}" | base64 -d)

To get the password for "dcache" run:

    export POSTGRES_PASSWORD=$(kubectl get secret --namespace dcache-test chimera-postgresql -o jsonpath="{.data.password}" | base64 -d)

To connect to your database run the following command:

    kubectl run chimera-postgresql-client --rm --tty -i --restart='Never' --namespace dcache-test --image docker.io/bitnami/postgresql:15.4.0-d

## List the running helm charts and k8s pods

In [19]:
helm -n ${K8S_NAMESPACE} list

NAME   	NAMESPACE  	REVISION	UPDATED                                 	STATUS  	CHART              	APP VERSION
billing	dcache-test	1       	2024-06-04 21:53:09.224625688 +0200 CEST	deployed	kafka-23.0.7       	3.5.1      
cells  	dcache-test	1       	2024-06-04 21:52:37.223519598 +0200 CEST	deployed	zookeeper-13.4.1   	3.9.2      
chimera	dcache-test	1       	2024-06-04 21:52:15.507075883 +0200 CEST	deployed	postgresql-12.12.10	15.4.0     


In [20]:
kubectl -n ${K8S_NAMESPACE} get pods

NAME                   READY   STATUS    RESTARTS   AGE
billing-kafka-0        1/1     Running   0          116s
cells-zookeeper-0      1/1     Running   0          2m28s
chimera-postgresql-0   1/1     Running   0          2m50s


## Deploy dCache

Starting version 9.2.0 dcache containers are available on docker hub.

This will start a pre-configured dCache cluster named `store`.

In [21]:
helm -n ${K8S_NAMESPACE} install --wait --set image.tag=10.0.3 store dcache/dcache

NAME: store
LAST DEPLOYED: Tue Jun  4 21:55:41 2024
NAMESPACE: dcache-test
STATUS: deployed
REVISION: 1
TEST SUITE: None


### Get list of running services for door and pool

In [23]:
kubectl -n ${K8S_NAMESPACE} get services -l=app=door

NAME             TYPE        CLUSTER-IP    EXTERNAL-IP   PORT(S)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               AGE
store-door-svc   ClusterIP   10.43.78.54   <none>        2049/TCP,22125/TCP,22128/TCP,1094/TCP,1095/TCP,8080/TCP,8083/TCP,2811/TCP,8443/TCP,22224/TCP,3880/TCP,11111/TCP,28000/TCP,28001/TCP,28002/TCP,28003/TCP,28004/TCP,28005/TCP,28006/TCP,28007/TCP,28008/TCP,28009/TCP,28010/TCP,28011/TCP,28012/TCP,28013/TCP,28014/TCP,28015/

In [24]:
kubectl -n ${K8S_NAMESPACE} get services -l=app=pool-a

NAME               TYPE        CLUSTER-IP     EXTERNAL-IP   PORT(S)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       AGE
store-pool-a-svc   ClusterIP   10.43.14.206   <none>        32049/TCP,31094/TCP,38080/TCP,38083/TCP,28000/TCP,28001/TCP,28002/TCP,28003/TCP,28004/TCP,28005/TCP,28006/TCP,28007/TCP,28008/TCP,28009/TCP,28010/TCP,28011/TCP,28012/TCP,28013/TCP,28014/TCP,28015/TCP,28016/TCP,28017/TCP,28018/TCP,28019/TCP,28020/TCP,28021/TCP,28022/TCP,28023/TCP,28024/TCP,28025/TCP,28026/TCP,28027/TCP,28028/TCP,2802

## Add a worker node

Add a worker that runs in the same namespace as the dCache services and
can be used to test the dCache services. The worker nod just sits there and does nothing :)


In [25]:
kubectl -n ${K8S_NAMESPACE} run wn --image almalinux:9 -- /bin/sleep inf

pod/wn created


### Access the worker node with exec and run some tests.

In [27]:
kubectl -n ${K8S_NAMESPACE} exec -ti wn -- curl -k --head https://store-door-svc:8083/data

HTTP/1.1 200 OK
Date: Tue, 04 Jun 2024 20:20:49 GMT
Server: dCache/10.0.3
Accept-Ranges: bytes
ETag: "0000569948F92CB14E18AFE7760C48F69956_-455864921"
Last-Modified: Tue, 04 Jun 2024 19:57:33 GMT
Link: <https://store-door-svc:8083/data?type=metalink>; rel=describedby; type="application/metalink4+xml"
Content-Type: text/html;charset=utf-8
Transfer-Encoding: chunked



## Some other staff that works in  terminal, but not in a notebook


### Access dCache admin interface

```bash
kubectl -n ${K8S_NAMESPACE} run --rm -ti admin-ssh --image kroniak/ssh-client -- /bin/bash
```


### An interactive WN session.
```bash
kubectl -n ${K8S_NAMESPACE} exec -ti wn -- /bin/bash
```


## Delete all and the end

In [28]:
kubectl delete namespace ${K8S_NAMESPACE}

namespace "dcache-test" deleted
